<a href="https://colab.research.google.com/github/sujeetgund/Langchain_rag/blob/main/Rag_on_Google_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain langchain_community langchain-google-genai langchain-groq

In [1]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")


## Model to generate response

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash")

## Model to create embeddings

In [17]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

## Vector Store to save documents

In [18]:
from langchain_community.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embedding)

## Web Based Document Loader

In [10]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_paths=[
        "https://www.financialexpress.com/money/cabinet-approves-incentive-scheme-for-low-value-upi-transactions-check-details-3781982/"
    ]
)

docs = loader.load()

In [ ]:
docs

## Splitting Documents into Chunks

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

len(chunks)

32

## Store Documents into Vector Store

In [19]:
_ = vector_store.add_documents(documents=chunks)

## Design Prompt

In [21]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate(
    template="""
    You are an AI-powered news assistant. You have been provided with relevant context from a vector database containing news articles. Your task is to generate a well-structured, human-like response that satisfies the user's query using the given context. Ensure the response is clear, engaging, and informative. Fill any gaps using general knowledge when appropriate, but do not speculate. Maintain a neutral and professional tone.

    Context: {context}
    User Query: {query}

    Provide a concise, factual, and user-friendly response.""",
    input_variables = ["context", "query"]
)

## Write Class for RAG App

In [27]:
class RAG:
    def __init__(self, model, vector_store):
      self.model = model
      self.vector_store = vector_store
      self.query = ""
      self.context = ""
      self.response = ""

    def retrieve(self, query: str) -> str:
      self.query = query
      self.context = self.vector_store.similarity_search(query, k=3)
      return self.context

    def generate(self) -> str:
      prompt = template.format(context=self.context, query=self.query)
      self.response = self.model.invoke(prompt).content
      return self.response


In [28]:
app = RAG(model, vector_store)

In [29]:
query = "Who will get incentive?"

app.retrieve(query)

[Document(id='981369df-80c8-4895-bbc1-9c5b5f28cfab', metadata={'source': 'https://www.financialexpress.com/money/cabinet-approves-incentive-scheme-for-low-value-upi-transactions-check-details-3781982/', 'title': 'Cabinet approves incentive scheme for low-value UPI transactions - Check details - Money News | The Financial Express', 'description': 'The incentive will be provided at the rate of 0.15% per transaction value for transactions up to Rs 2,000 pertaining to category of small merchants, the government said in an official release.', 'language': 'en'}, page_content='Funds       Business NewsmoneyCabinet approves incentive scheme for low-value UPI transactions - Check details  Cabinet approves incentive scheme for low-value UPI transactions – Check details The incentive will be provided at the rate of 0.15% per transaction value for transactions up to Rs 2,000 pertaining to category of small merchants, the government said in an official release.'),
 Document(id='d53fb5fc-aa1a-4178-9

In [30]:
app.generate()

'The incentive will be provided to small merchants for low-value UPI transactions up to ₹2,000. The incentive is set at 0.15% per transaction value.'